# 앙상블 학습
여러 모델로부터 예측을 수집, 일련의 예측기를 앙상블

### 랜덤 포레스트
결정 트리의 앙상블


### 투표 기반 분류기
분류기 여러개를 훈련시킨 모델을 다수결 투표로 분류 : 직접 투표 분류기

앙상블 기법은 A, B 클래스 중 A 확률이 나올 확률이 51퍼센트인 어떤 이진 분류기들 1000개를 구축했을 때, 75퍼센트의 정확도를 기대할 수 있다. 
(정확도가 51%에서 75%로 증가함)

수학적 원리는 이항분포와 확률 질량함수로 계산할 수 있다. 

그러나 분류기들이 대부분 같은 오류를 만들어 내기 때문에 앙상블의 정확도가 낮아진다. 
앙상블 기법은 각각의 모델들이 서로 독립적일 때 최고의 성능을 발휘한다. 

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voating_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting= 'hard'
)
voating_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voating_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.88
SVC 0.896
VotingClassifier 0.904


### Bagging & Pasting

Bagging : 훈련 세트에서 중복을 허용하여 샘플링하는 방식

Pasting : 훈련 세트에서 중복을 허용하지 않고 샘플링하는 방식

배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링할 수 있다.